## Двухфакторный дисперсионный анализ

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [109]:
df = pd.read_csv('atherosclerosis.csv')

In [42]:
df.sample(5)

,expr,age,dose
44,110.833715,1,D2
18,94.290992,2,D1
2,103.435134,1,D1
38,109.626718,1,D2
22,97.030269,2,D1


In [43]:
table = pd.DataFrame(columns=['Age', 'Dose','N', 'M', 'SD'])

In [44]:
table['Age'] = ['Young', 'Old','Young', 'Old']

In [45]:
table.loc[table.index == 0, 'Dose'] ='High'
table.loc[table.index == 1, 'Dose'] ='High'
table.loc[table.index == 2, 'Dose'] ='Low'
table.loc[table.index == 3, 'Dose'] ='Low'

In [62]:
table.loc[(table.Age == 'Young') & (table.Dose == 'High') , 'N'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].count()

In [63]:
table.loc[(table.Age == 'Young') & (table.Dose == 'Low') , 'N'] = df[(df['dose']=='D1') & (df['age'] == 1)]['expr'].count()

In [64]:
table.loc[(table.Age == 'Old') & (table.Dose == 'High') , 'N'] = df[(df['dose']=='D2') & (df['age'] == 2)]['expr'].count()

In [65]:
table.loc[(table.Age == 'Old') & (table.Dose == 'Low') , 'N'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].count()

In [66]:
table.loc[(table.Age == 'Young') & (table.Dose == 'High') , 'M'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].median()

In [67]:
table.loc[(table.Age == 'Young') & (table.Dose == 'Low') , 'M'] = df[(df['dose']=='D1') & (df['age'] == 1)]['expr'].median()

In [68]:
table.loc[(table.Age == 'Old') & (table.Dose == 'High') , 'M'] = df[(df['dose']=='D2') & (df['age'] == 2)]['expr'].median()

In [69]:
table.loc[(table.Age == 'Old') & (table.Dose == 'Low') , 'M'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].median()

In [71]:
table.loc[(table.Age == 'Young') & (table.Dose == 'High') , 'SD'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].std()

In [72]:
table.loc[(table.Age == 'Young') & (table.Dose == 'Low') , 'SD'] = df[(df['dose']=='D1') & (df['age'] == 1)]['expr'].std()

In [73]:
table.loc[(table.Age == 'Old') & (table.Dose == 'High') , 'SD'] = df[(df['dose']=='D2') & (df['age'] == 2)]['expr'].std()

In [74]:
table.loc[(table.Age == 'Old') & (table.Dose == 'Low') , 'SD'] = df[(df['dose']=='D2') & (df['age'] == 1)]['expr'].std()

In [76]:
table.head()

,Age,Dose,N,M,SD
0,Young,High,16,106.551,4.36902
1,Old,High,16,101.875,5.13537
2,Young,Low,16,104.779,5.86345
3,Old,Low,16,106.551,4.36902


In [133]:
def two_factor_dispersion(df, first_factor, second_factor, depended_factor):
    # Calculate count
    N = df[first_factor].count()
    m_a = df[first_factor].nunique()
    m_b = df[second_factor].nunique()
    # Calculate degrees of freedom
    df_total = N-1
    df_a = m_a - 1
    df_b = m_b - 1
    df_residuals = df_total - (df_a + df_b) 
    df_mean = df[depended_factor].mean()
    # Calculate sum of squares
    SSW = np.sum((elem - df_mean)**2 for elem in df[depended_factor])
    SSBa = np.sum([(df[df[first_factor] == ff_cat][depended_factor].mean()-df_mean)**2 for ff_cat in df[first_factor]])
    SSBb = np.sum([(df[df[second_factor] == sf_cat][depended_factor].mean() - df_mean)**2 for sf_cat in df[second_factor]])
    SStotal = SSW + SSBa + SSBb + (SSBa*SSBb)
    # Calculate mean square
    MSA = SSBa/df_a
    MSB = SSBb/df_b
    MSW = SSW/df_residuals
    # Calcaulte f-value
    FA = MSA/MSW
    FB = MSB/MSW
    # Calculate p-value
    p_a = stats.f.sf(FA, df_a, df_residuals)
    p_b = stats.f.sf(FB, df_b, df_residuals)
    # Combine values to one table
    table = pd.DataFrame(columns = ['Factors', 'DF', 'Sum sq', 'Mean sq', 'f-value', 'p-value'])
    table['Factors'] = [first_factor, second_factor, 'Residuals']
    table.loc[table['Factors'] ==first_factor,'DF'] = df_a
    table.loc[table['Factors'] == second_factor,'DF'] = df_b
    table.loc[table['Factors'] =='Residuals','DF'] = df_residuals
    table.loc[table['Factors'] == first_factor,'Sum sq'] = SSBa
    table.loc[table['Factors'] == second_factor,'Sum sq'] = SSBb
    table.loc[table['Factors'] =='Residuals','Sum sq'] = SSW
    table.loc[table['Factors'] == first_factor,'Mean sq'] = MSA
    table.loc[table['Factors'] == second_factor,'Mean sq'] = MSB
    table.loc[table['Factors'] =='Residuals','Mean sq'] = MSW
    table.loc[table['Factors'] == first_factor,'f-value'] = FA
    table.loc[table['Factors'] == second_factor,'f-value'] = FB
    table.loc[table['Factors'] == first_factor,'p-value'] = p_a
    table.loc[table['Factors'] == second_factor,'p-value'] = p_b
    return table
two_factor_dispersion(df,'age', 'dose', 'expr')

C:\Users\Diana_Aimbetova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  del sys.path[0]


,Factors,DF,Sum sq,Mean sq,f-value,p-value
0,age,1,197.453,197.453,6.67089,0.0122143
1,dose,1,16.9122,16.9122,0.571375,0.452622
2,Residuals,61,1805.55,29.5992,NaN,NaN


In [134]:
df_2 = pd.read_csv('birds.csv')

In [135]:
df_2.head()

,var4,hormone,sex
0,17.859039,1,1
1,20.842343,1,1
2,19.318099,1,1
3,20.064451,1,1
4,17.620316,1,1


In [136]:
two_factor_dispersion(df_2,'hormone', 'sex', 'var4')

C:\Users\Diana_Aimbetova\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  del sys.path[0]


,Factors,DF,Sum sq,Mean sq,f-value,p-value
0,hormone,1,0.847472,0.847472,0.0762361,0.783398
1,sex,1,0.119762,0.119762,0.0107735,0.917672
2,Residuals,61,678.101,11.1164,NaN,NaN
